In [1]:
import graphlab as gl
# set canvas to show sframes and sgraphs in ipython notebook
gl.canvas.set_target('ipynb')
import matplotlib.pyplot as plt
from PodcastFeatureEngineering import Podcast_Features
%matplotlib inline
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

2016-04-17 15:23:43,048 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1460906622.log


This non-commercial license of GraphLab Create is assigned to crystal.humphries@systemsbiology.org and will expire on November 04, 2016. For commercial licensing options, visit https://dato.com/buy/.


In [10]:
train_file = 'http://s3.amazonaws.com/dato-datasets/millionsong/10000.txt'

# # The below will download a 118 MB file.
sf = gl.SFrame.read_csv(train_file, header=False, delimiter='\t', verbose=False)
# sf.rename({'X1':'user_id', 'X2':'song_id', 'X3':'listen_count'}).show()

In [ ]:
(train_set, test_set) = sf.random_split(0.8, seed=1)

In [ ]:
popularity_model = gl.popularity_recommender.create(train_set, 'user_id', 'song_id')

In [2]:
items = gl.load_sframe("items_podcast_sframe.gl/")

In [ ]:
items.head()

In [ ]:
a = gl.item_similarity_recommender.create(items, user_id="TotalEpisodes", item_id="Title", target="CategoryRanking")

In [ ]:
PF = Podcast_Features()
df = PF.cleaned_df

In [ ]:
dates=[]
for a,b in  zip(PF.df.DateLastEpisode, PF.df.DateFirstEpisode):
    try:
        new = (a-b).days
    except:
        new = 'NaT'
    dates.append(new)
dates = np.array(dates)

In [ ]:
g = df.columns.to_series().groupby(df.dtypes).groups
c = g.keys()
df2 = df.drop(g[c[0]], axis=1)
df2 = df2.drop(g[c[1]], axis=1)

In [ ]:
df2['Title'] = df.Title
df2['dates'] = dates

In [ ]:
def get_top_100(col, df):
    tfidf = TfidfVectorizer( max_features=100, stop_words='english')
    X = tfidf.fit_transform(df[col])
    return pd.DataFrame(X.toarray(),
                        columns=pd.Series(tfidf.get_feature_names()))

In [ ]:
iTunes_Summary = get_top_100("iTunesSummary", df)

In [ ]:
df.info()

In [ ]:
df2['dates'] = df2['dates'].apply(lambda x: -1 if x=='NaT' else x)
df2['dates'] = df2['dates'].astype('int')

In [ ]:
df3 = df2.copy()

for col in ['iTunesSummary', 'iTunesKeywords', 'Category']:
    y = get_top_100(col, df)
    df3 = pd.concat([df3, y], axis=1)

In [ ]:
#np.duplicates(df3.columns)
from collections import Counter

In [ ]:
ls = Counter(df3.columns)

In [ ]:
df_final = pd.DataFrame()
for a,b in ls.items():
    if b==1:
        df_final = pd.concat([df_final, df3[a]],axis=1)
    else:
        df_final[a] = df3[[a]].sum(axis=1)

In [ ]:
sum_ar = df_final.summaries.unique()

In [ ]:
sf = gl.SFrame(df_final)

In [26]:
df_final = pd.read_pickle("item_all.pkl")
df_num = df_final.drop(["Title", "summaries"], axis=1)

In [ ]:
items = pd.read_json('random')

In [3]:
users = gl.SFrame.read_csv('random1_t')

Finished parsing file /home/ubuntu/PodcastRecommender/scripts/random1_t

Parsing completed. Parsed 100 lines in 0.59425 secs.

Read 1414 lines. Lines per second: 699.928

Read 5658 lines. Lines per second: 754.045

Read 9902 lines. Lines per second: 786.194

Finished parsing file /home/ubuntu/PodcastRecommender/scripts/random1_t

Parsing completed. Parsed 11071 lines in 14.2319 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int

In [ ]:
sf1 = sf[sf['summaries']==sum_ar[0]]

In [4]:
names = pd.read_pickle("users_10orMorePodcst.pkl")
names = gl.SArray(names.index.tolist())
#users.add_column(names, 'Title')

In [23]:
users = gl.load_sframe('df_user_info.gl')

In [24]:
users.show()

In [8]:
users.head()

X1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
users.shape

(11071, 18530)

In [ ]:
main_df = sf1[['TotalEpisodes','Title','summaries']]

In [ ]:
item_desccription = sf1.remove_column('TotalEpisodes')

In [43]:
a = users['user_id']

In [36]:
ls = ['user_id_']*len(a)

In [45]:
users['user_id'] = gl.SFrame([ z+str(int(b)) for z,b in zip(ls,a)])

TypeError: Unexpected data source. Possible data source types are: list, numpy.ndarray, pandas.Series, and string(url)

In [48]:
t = gl.SArray([ z+str(int(b)) for z,b in zip(ls,a)])

In [50]:
users['user_id'] = t

In [51]:
a = gl.item_similarity_recommender.create(users, user_id="user_id", item_id="Title", 
                                           item_data=items)

Recsys training: model = item_similarity

Preparing data set.

Data has 408740 observations with 11071 users and 152883 items.

Data prepared in: 1.71879s

Processing 408740 observations:

...

5     13879   3.07119

6     16164   3.16304

7     18524   3.25672

Finished getting similar items for 152883 items in 3.25687s.

Finished training in 3.94994s

In [53]:
a.get_similar_items()

Getting similar items completed in 0.099298

Title,similar,score,rank
The History of Awesome -IGN Presents ...,IGN.com - Daily Fix(Video) ...,1.0,1
The History of Awesome -IGN Presents ...,IGN Games Podcasts,1.0,2
The History of Awesome -IGN Presents ...,IGN.com - IGN_Strategize(Video) ...,1.0,3
The History of Awesome -IGN Presents ...,IGN.com - Game Scoop! TV(Video) ...,1.0,4
The History of Awesome -IGN Presents ...,Themes for Beginners ofFrench and Travel – ...,0.333333333333,5
The History of Awesome -IGN Presents ...,USCHO Live!,0.333333333333,6
The History of Awesome -IGN Presents ...,Thank You Ocean »Podcast Feed ...,0.333333333333,7
The History of Awesome -IGN Presents ...,Kotaku Talk Radio,0.333333333333,8
The History of Awesome -IGN Presents ...,Your Voice For Hope,0.25,9
The History of Awesome -IGN Presents ...,FirstFT: The World in 60Seconds (Video) ...,0.25,10


In [ ]:
##need to do this!
percentage_english = df.iTunesSummary.apply(lambda x: pods._find_percentage(x))
df2 = df[percentage_english>=0.6]

In [ ]:
summaries = PF.get_new_column_of_text('iTunesKeywords')

NameError: name 'df3' is not defined

In [29]:
items = gl.SFrame(df_final)

In [ ]:
df_final['summaries']=pd.Series([' '.join(x) for x in summaries[0]])
#sum_ar

In [ ]:
gl.item_similarity_recommender??